In [2]:
import requests
import json
import pandas as pd
import sys 
from datetime import datetime
from dateutil.relativedelta import relativedelta
from getpass import getpass
from mysql.connector import connect, Error
import re
import os
import time
import shutil

with connect(
    host="localhost",
    user="root",
    password="Gtxtymr@21",
    database='daily_operations'

) as connection:
    
    max_date = pd.read_sql('SELECT MAX(operation_date) FROM voo_operations', connection).iloc[0, 0]

if max_date >= (datetime.today() + relativedelta(days=-2)).date():
    pass
else:
    DayStart = (max_date + relativedelta(days=1)).strftime('%Y-%m-%d')
    DayEnd = (datetime.today() + relativedelta(days=-2)).strftime('%Y-%m-%d')

    
    path = r'D:\FTP-DATA\DB_download\\'

    dates_order = [datetime.strftime(i, '%d.%m.%Y') for i in pd.date_range(DayStart, DayEnd).to_list()]

    voo_op = pd.DataFrame()
    voo_clients = pd.DataFrame()

    op_cols = ['Дата', 'id операции', 'Отделение', 'Тип_справочника', 'id клиента', 'Дата и время', 'Тип операции', 'Валюта операции', 
               'Сумма операции, руб.', 'Доход операции, руб.', 'Курс покупки', 'Курс продажи', 'Акция', 'Скидка с курса', 'Номер карты лояльности',
               'Начислено баллов за операцию (если есть карта)', 'Баланс баллов на карте']

    clients_cols = ['Дата', 'id клиента', 'Тип_справочника','Пол', 'Возраст', 'Номер телефона', 'e-mail', 'Адрес клиента', 
                    'Номер карты лояльности']

    for date in dates_order:
        year_path_name = path + [file for file in os.listdir(path) if all([date[-4:] in file, '$' not in file, os.path.isdir(path + file)])][0]
        for file in os.listdir(path):
            if 'voo' in file.lower() and 'op' in file.lower() and '$' not in file and date in file and 'csv' in file:
                voo_op = voo_op.append(pd.read_csv(path + file, sep=';', names=op_cols, engine='python'))
                shutil.move(path + file, year_path_name)
            if 'voo' in file.lower() and 'clients' in file.lower() and '$' not in file and date in file and 'csv' in file:
                voo_clients = voo_clients.append(pd.read_csv(path + file, sep='|', names=clients_cols, engine='python'))
                shutil.move(path + file, year_path_name)


    def to_float(x):
        if pd.isnull(x):
            return x
        if isinstance(x, str):
            x = x.replace(',', '.')
            return float(x)
        else:
            return float(x)

    def to_int(x):
        if pd.isnull(x):
            return x
        if isinstance(x, str):
            x = x.replace(',', '.')
            return int(float(x))
        else:
            return int(x) 

    def right_phone_card(x):
        if pd.isnull(x):
            return x    
        x = str(x)
        return x


    voo_op['Дата'] = pd.to_datetime(voo_op['Дата'], format='%d.%m.%Y').dt.date
    voo_op['id операции'] = voo_op['id операции'].apply(to_int)
    voo_op['id клиента'] = voo_op['id клиента'].apply(to_int)
    voo_op['Дата и время'] = pd.to_datetime(voo_op['Дата и время'], format='%d.%m.%Y %H:%M:%S')
    voo_op['Сумма операции, руб.'] = voo_op['Сумма операции, руб.'].apply(to_float)
    voo_op['Доход операции, руб.'] = voo_op['Доход операции, руб.'].apply(to_float)
    voo_op['Курс покупки'] = voo_op['Курс покупки'].apply(to_float)
    voo_op['Курс продажи'] = voo_op['Курс продажи'].apply(to_float)
    voo_op['Скидка с курса'] = voo_op['Скидка с курса'].apply(to_float)
    voo_op['Номер карты лояльности'] = voo_op['Номер карты лояльности'].apply(right_phone_card)
    voo_op['Начислено баллов за операцию (если есть карта)'] = voo_op['Начислено баллов за операцию (если есть карта)'].apply(to_int)
    voo_op['Баланс баллов на карте'] = voo_op['Баланс баллов на карте'].apply(to_int)

    voo_clients['Дата'] = pd.to_datetime(voo_clients['Дата'], format='%d.%m.%Y').dt.date
    voo_clients['id клиента'] = voo_clients['id клиента'].apply(to_int)
    voo_clients['Возраст'] = voo_clients['Возраст'].apply(to_int)
    voo_clients['Номер телефона'] = voo_clients['Номер телефона'].apply(right_phone_card)
    voo_clients['Номер карты лояльности'] = voo_clients['Номер карты лояльности'] .apply(right_phone_card)     

    voo_op = voo_op.where(pd.notnull(voo_op), None)
    voo_clients = voo_clients.where(pd.notnull(voo_clients), None)

    voo_op_sql = [tuple(row) for index, row in voo_op.iterrows()]
    voo_clients_sql = [tuple(row) for index, row in voo_clients.iterrows()]   

    with connect(
        host="localhost",
        user="root",
        password='Gtxtymr@21',
        database='daily_operations',
        connection_timeout=28800

    ) as connection:
        query = """INSERT INTO voo_operations
        (operation_date,
        operation_id, 
        department, 
        handbook_type,
        client_id,
       operation_datetime, 
       operation_type, 
       currency_type,
       operation_sum, 
       operation_revenue, 
       purchase_rate, 
       selling_rate, 
       marketing_campaign,
       discount_course,
       card_number,
       bonus_income,
       card_balance)
       VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        with connection.cursor() as cursor:
            for i in range(0, len(voo_op_sql), 2):
                j = i + 2
                if j > len(voo_op_sql):
                    j = len(voo_op_sql)
                try:
                    cursor.executemany(query, voo_op_sql[i:j])
                    connection.commit()
                except:
                    continue

    time.sleep(1)
    
    with connect(
        host="localhost",
        user="root",
        password='Gtxtymr@21',
        database='daily_operations',
        connection_timeout=28800

    ) as connection:
        query = """INSERT INTO voo_clients
       (client_date,
        client_id,  
        handbook_type,
        client_gender,
       client_age,
       client_phone,
       client_email, 
       client_adress,
       card_number)
       VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        with connection.cursor() as cursor:
            for i in range(0, len(voo_clients_sql), 2):
                j = i + 2
                if j > len(voo_clients_sql):
                    j = len(voo_clients_sql)
                try:    
                    cursor.executemany(query, voo_clients_sql[i:j])
                    connection.commit()
                except:
                    continue










IndexError: list index out of range

In [15]:


    
path = r'D:\FTP-DATA\DB_download\\'

dates_order = [datetime.strftime(i, '%d.%m.%Y') for i in pd.date_range(DayStart, DayEnd).to_list()]


[file for file in os.listdir(path) if all([date[-4:] in file, '$' not in file, os.path.isdir(path + file)])]

['Операции_ВОО_за_2021_год']

In [13]:
for file in os.listdir(path):
    if dates_order[0][-4:] in file and '$' not in file:
        print(os.path.isdir(path + file))

VOO_CLIENTS_11.05.2021_11.05.2021.csv
VOO_OP_11.05.2021_11.05.2021.csv
Операции_ВОО_за_2021_год
